# CityBikes

In [1]:
import pandas as pd
import json

In [7]:
# API Base URL 
base_api = 'http://api.citybik.es/v2/networks'
# Read the json
data = pd.read_json(base_api)
# Store formated data in DataFrame
networks = pd.json_normalize(data['networks'])
# Display a sample of the data
print(networks.head(1))
# Show networks ID & City in Taiwan
print(networks[['id', 'location.city']][networks['location.country'] == 'TW'].reset_index(drop=True))

                      id                   name  \
0  abu-dhabi-careem-bike  Abu Dhabi Careem BIKE   

                                 href   company  \
0  /v2/networks/abu-dhabi-careem-bike  [Careem]   

                                           gbfs_href  location.latitude  \
0  https://dubai.publicbikesystem.net/customer/gb...            24.4866   

   location.longitude location.city location.country system source ebikes  \
0             54.3728     Abu Dhabi               AE    NaN    NaN    NaN   

  license.name license.url scooters instances  
0          NaN         NaN      NaN       NaN  
                              id                  location.city
0            youbike-chiayi-city                 台灣 Chiayi City
1           youbike-hsinchu-city               新竹市 Hsinchu City
2         youbike-hsinchu-county             新竹縣 Hsinchu County
3   youbike-hsinchu-science-park  新竹科學工業園區 Hsinchu Science Park
4         youbike-kaohsiung-city             高雄市 Kaohsiung City
5       

Send a request to CityBikes for the city of your choice. 

In [8]:
# Bike Network in Tainan, Taiwan
network_id = 'youbike-tainan-city'
# Get data fron API
data = pd.read_json(base_api + f'/{network_id}?fields=stations')
# Filter the data
stations = data['network']['stations'].copy()
# stations

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [4]:
# Save all stations info in a list
stations_list = []
for i, station in enumerate(stations):
    lat = f"{station['latitude']:.6f}"
    lon = f"{station['longitude']:.6f}"
    free = station['free_bikes']
    bikes = 'bike' if free == 1 else 'bikes'
    # print(station['name'])
    # print(f"{i + 1}. ({lat}, {lon}) : {free} {bikes} available")
    # if lat == '43.6532': print(station['name'])
    stations_list.append(
        {
            'latitude': float(lat),
            'longitude': float(lon),
            'bikes_available': int(free)
        },
    )

Put your parsed results into a DataFrame.

In [5]:
# Store the list in a DataFrame
stations_df = pd.DataFrame(stations_list)
# Print top 5 station with most bikes available
print(stations_df.sort_values(by='bikes_available', ascending=False).head())
# DataFrame Info
stations_df.info()

      latitude   longitude  bikes_available
482  22.992150  120.231230               32
95   23.003997  120.200593               31
158  22.989290  120.231160               30
135  22.922310  120.227280               29
496  23.133640  120.305990               29
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525 entries, 0 to 524
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   latitude         525 non-null    float64
 1   longitude        525 non-null    float64
 2   bikes_available  525 non-null    int64  
dtypes: float64(2), int64(1)
memory usage: 12.4 KB


In [6]:
# Saving data into CSV file
stations_df.to_csv('../data/city_bikes.csv', index=False)